In [ ]:
# import pandas as pd
# 
# path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
# complete_df = pd.read_csv(path + 'cleaned_strings_df.csv')
# df = complete_df.iloc[80000:90000]
# df = df.loc[~df['stanzas'].isna()]

In [7]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
# ! replace with your file path
df = pd.read_csv(path + 'bruno1.csv')

In [8]:
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor

# Load the BART large MNLI model for zero-shot classification
classifier = pipeline(
    "zero-shot-classification",
    model="albert-base-v2"
)

emotion_labels = [
    "anger", "anticipation", "joy", "trust",
    "fear", "surprise", "sadness", "disgust"
]

def classify_emotion(text):
    # print(text)
    result = classifier(text, candidate_labels= emotion_labels)
    return result['labels'][0]  # Return the top label

def classify_emotions_in_parallel(texts, max_workers=4):
    # Using ThreadPoolExecutor to process texts in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(classify_emotion, texts))
    return results

# with Pool(processes= 4) as pool:
#     print(pool.map)

# classify_emotion(df.loc[0, 'stanzas'])

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [9]:
# classify_emotion(df.loc[1, 'stanzas'])

In [10]:
# classify_emotions_in_parallel(df['stanzas'].head(n= 1000))

In [11]:
df

,Unnamed: 0,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanzas,stanza_number,is_chorus
0,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"Oops Now! (Wait for me! Me, me..",1,False
1,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"Oops Now! (I can feel it, feel it, feel it...",2,True
2,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,Oops Now! Do Di Do Di Dam,3,True
3,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"As I feel so far away from me, (Way from meI w...",4,False
4,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,Oops Now! Do di Do Di Dam,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,15437,PRETTY,Kennie J.D.,2020,978,{''},False,True,False,False,False,Does she know you call me after hours While s...,4,True
9995,15437,PRETTY,Kennie J.D.,2020,978,{''},False,True,False,False,False,"Red dress, red everything Black silk, what yo...",5,False
9996,15438,Crawling Back,Alex Goot,2013,65,{''},False,True,False,False,False,I tried to leave you once but I could not get ...,0,False
9997,15439,Jersey,Naive New Beaters,2015,454,{''},False,True,False,False,False,Riding Cadillacs just like Dr Dre Dre Dre Taki...,0,False


In [12]:
# Apply the classification to each row in the dataframe
df['label'] = classify_emotions_in_parallel(df['stanzas'])
df

,Unnamed: 0,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanzas,stanza_number,is_chorus,label
0,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"Oops Now! (Wait for me! Me, me..",1,False,joy
1,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"Oops Now! (I can feel it, feel it, feel it...",2,True,anticipation
2,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,Oops Now! Do Di Do Di Dam,3,True,surprise
3,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,"As I feel so far away from me, (Way from meI w...",4,False,joy
4,13738,Oops Do Di Do Di Dam,Sugar ft Millie,2019,185,{''},False,True,False,False,False,Oops Now! Do di Do Di Dam,5,True,surprise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,15437,PRETTY,Kennie J.D.,2020,978,{''},False,True,False,False,False,Does she know you call me after hours While s...,4,True,fear
9995,15437,PRETTY,Kennie J.D.,2020,978,{''},False,True,False,False,False,"Red dress, red everything Black silk, what yo...",5,False,joy
9996,15438,Crawling Back,Alex Goot,2013,65,{''},False,True,False,False,False,I tried to leave you once but I could not get ...,0,False,anticipation
9997,15439,Jersey,Naive New Beaters,2015,454,{''},False,True,False,False,False,Riding Cadillacs just like Dr Dre Dre Dre Taki...,0,False,joy


In [13]:
# !replace with <path>/<tuo_nome>_labelled.csv
df.to_csv(path + 'bruno1_labelled.csv', index= False)

In [1]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
lemmatized_df = pd.read_csv(path + 'def_lemmatized_df.csv')

In [2]:
lemmatized_df

,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanza_number,is_chorus,lemmatized_stanzas
0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,0,False,"['she', 'get', 'a', 'broke', 'down', 'el', 'ca..."
1,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,1,True,"['something', 'in', 'the', 'water', 'something..."
2,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,2,False,"['she', 'do', 'her', 'makeup', 'and', 'hair', ..."
3,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,4,False,"['my', 'hoosi', 'girl', 'be', 'so', 'fine', 's..."
4,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,5,True,"['something', 'in', 'the', 'water', 'something..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
869764,Plunderberg,Lagerstein,2012,3,{''},False,False,False,False,True,3,False,"['set', 'sail', 'full', 'steam', 'ahead', 'to'..."
869765,Plunderberg,Lagerstein,2012,3,{''},False,False,False,False,True,4,False,"['and', 'as', 'you', 'stare', 'into', 'the', '..."
869766,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,0,True,"['the', 'thin', 'gruel', 'for', 'million', 'of..."
869767,The Thin Gruel,Buck Gooter,2012,28,{''},False,False,False,False,True,1,False,"['nothing', 'be', 'leave', 'but', 'a', 'mounta..."


# Save the new file

In [2]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
lemmatized_df = pd.read_csv(path + 'def_lemmatized_df.csv')
labelled_df = pd.concat(
    [
        pd.read_csv(path + 'bruno_labelled.csv'),
        pd.read_csv(path + 'gaia_labelled.csv'),
        pd.read_csv(path + 'francesco_labelled.csv'),
        pd.read_csv(path + 'lavinia_labelled.csv'),
        pd.read_csv(path + 'bruno1_labelled.csv'),
        pd.read_csv(path + 'gaia1_labelled.csv'),
        pd.read_csv(path + 'lavinia1_labelled.csv'),
        pd.read_csv(path + 'francesca_labelled.csv'),
        pd.read_csv(path + 'francesco1_labelled.csv'),
    ], ignore_index= True
)
labelled_df

,Unnamed: 0,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanzas,stanza_number,is_chorus,label
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She got a broke down El Camino, in the front y...",0,False,anger
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,1,True,anger
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She does her makeup and hair, to cook fried ch...",2,False,anticipation
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,3,True,anger
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"My hoosier girl is so fine, shake the watermel...",4,False,fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119990,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,(KJ-52 Tired of those CD's that don't leave yo...,0,False,surprise
119991,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,{Todd Collins in weird voice} That's right it ...,1,False,anticipation
119992,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,"(KJ-52 But don't take our word for it, ask the...",2,False,sadness
119993,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,I used to be a cross eyed bowlegged one-armed ...,3,False,joy


In [3]:
labelled_df.rename(
    columns= {'Unnamed: 0' : 'id'},
    inplace= True
)
labelled_df

,id,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanzas,stanza_number,is_chorus,label
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She got a broke down El Camino, in the front y...",0,False,anger
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,1,True,anger
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"She does her makeup and hair, to cook fried ch...",2,False,anticipation
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,Something in the water (something in the wate...,3,True,anger
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,"My hoosier girl is so fine, shake the watermel...",4,False,fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119990,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,(KJ-52 Tired of those CD's that don't leave yo...,0,False,surprise
119991,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,{Todd Collins in weird voice} That's right it ...,1,False,anticipation
119992,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,"(KJ-52 But don't take our word for it, ask the...",2,False,sadness
119993,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,I used to be a cross eyed bowlegged one-armed ...,3,False,joy


In [4]:
df = pd.merge(
    lemmatized_df, labelled_df[
        ['id', 'stanza_number', 'label']
    ],
    on= ['id', 'stanza_number'], how= 'inner'
)
df

,id,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanza_number,is_chorus,lemmatized_stanzas,label
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,0,False,"['she', 'get', 'a', 'broke', 'down', 'el', 'ca...",anger
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,1,True,"['something', 'in', 'the', 'water', 'something...",anger
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,2,False,"['she', 'do', 'her', 'makeup', 'and', 'hair', ...",anticipation
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,4,False,"['my', 'hoosi', 'girl', 'be', 'so', 'fine', 's...",fear
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,5,True,"['something', 'in', 'the', 'water', 'something...",fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108255,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,0,False,"['kj-52', 'tired', 'of', 'those', 'cd', ""'s"", ...",surprise
108256,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,1,False,"['todd', 'collin', 'in', 'weird', 'voice', 'th...",anticipation
108257,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,2,False,"['kj-52', 'but', 'do', 'not', 'take', 'our', '...",sadness
108258,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,3,False,"['I', 'use', 'to', 'be', 'a', 'cross', 'eye', ...",joy


In [5]:
df.to_csv(path + 'lab_lem_merge.csv', index= False)

In [6]:
df

,id,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanza_number,is_chorus,lemmatized_stanzas,label
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,0,False,"['she', 'get', 'a', 'broke', 'down', 'el', 'ca...",anger
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,1,True,"['something', 'in', 'the', 'water', 'something...",anger
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,2,False,"['she', 'do', 'her', 'makeup', 'and', 'hair', ...",anticipation
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,4,False,"['my', 'hoosi', 'girl', 'be', 'so', 'fine', 's...",fear
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,5,True,"['something', 'in', 'the', 'water', 'something...",fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108255,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,0,False,"['kj-52', 'tired', 'of', 'those', 'cd', ""'s"", ...",surprise
108256,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,1,False,"['todd', 'collin', 'in', 'weird', 'voice', 'th...",anticipation
108257,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,2,False,"['kj-52', 'but', 'do', 'not', 'take', 'our', '...",sadness
108258,22140,Infomercial Interlude,KJ-52,2003,67,{''},False,True,False,False,False,3,False,"['I', 'use', 'to', 'be', 'a', 'cross', 'eye', ...",joy
